In [1]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import src.soporte_api as sa
import src.soporte_limpieza as sl
import src.soporte_escrapeo as se
import os
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

In [3]:
ARCHIVO_RAW= os.getenv("ARCHIVO_RAW")
ARCHIVO_SALIDA = os.getenv("ARCHIVO_SALIDA")


In [4]:
df_raw=pd.read_parquet(ARCHIVO_RAW) #importo el fichero original
df=df_raw.copy() #Hago una copia del df original para seguir trabajando con él
df = df.drop_duplicates() #Elimino los duplicados

#Convierto las columnas en formato fecha
col_fechas=["fecha_reserva","inicio_estancia","final_estancia"]
sl.tran_data(col_fechas, df)

In [5]:
#Hay tanto clientes con ids duplicados como ids asignados a diferentes clientes. Por tanto, creo un id para cada cliente basandome la unión del nombre y apellido que es lo mismo que el correo electronico
df["nombre_apellido"] = df['nombre'].str.lower() +'.' +df['apellido'].str.lower()
df.nombre_apellido.unique()
df_clientes_unicos = pd.DataFrame(df.nombre_apellido.unique())
df_clientes_unicos['id_cliente_unico'] = None
for indice in df_clientes_unicos.index:
    df_clientes_unicos.iloc[indice,1] = 'C'+str(indice)+df_clientes_unicos.iloc[indice,0].split('.')[0][0]+df_clientes_unicos.iloc[indice,0].split('.')[1][0]
df_clientes_unicos=df_clientes_unicos.set_axis(['nombre_apellido', 'id_cliente_unico'], axis=1)

In [6]:
sl.incorporar_información_df_original(df,df_clientes_unicos, 'nombre_apellido', 'id_cliente_unico', 'id_cliente_unico')
df

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_cliente_unico
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,feliciana.cantón,C0fc
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,leonardo.hierro,C1lh
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,maite.calatayud,C2mc
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,tecla.bonet,C3tb
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,amílcar.andrés,C4aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,58acd6f3-7160-4973-b951-b14188937444,de729428-ec23-4665-a5c9-79e3ec282470,Lorena,Román,lorena.román@example.com,False,2025-02-06,2025-03-01,2025-03-02,35,111.78,Hotel Jardines del Rey,1.0,Madrid,lorena.román,C14900lr
14996,aa092c3e-7fe6-4985-a5a8-cacd38554b1b,8469b819-d96a-4904-9cdd-710764f6301f,Toño,Narváez,toño.narváez@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,,toño.narváez,C14901tn
14997,af0f6afa-e7d2-410a-8cdd-5eb62b1e4788,e89751f4-3621-4636-81d3-90e2baa709cb,Nadia,Verdugo,nadia.verdugo@example.com,False,2025-02-11,2025-03-01,2025-03-02,47,442.96,Hotel Brisas del Mar,5.0,Madrid,nadia.verdugo,C14902nv
14998,6d1f590c-ae87-4582-bc99-9b8cbf1d8e28,c4117358-fd89-44cd-b388-4ec6d3538c72,Custodia,Marco,custodia.marco@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,custodia.marco,C14903cm


In [7]:
#Relleno los valores nulos de las columnas inicio estancia y final estancia con el único valor existente:
df.inicio_estancia.value_counts()
df["inicio_estancia"] = df[df["inicio_estancia"].notna()].inicio_estancia[0]
df["final_estancia"] =  df[df["final_estancia"].notna()].final_estancia[0]


In [8]:
#Creo un diccionario nuevo para darle un id a cada hotel propio según el nombre:
df_no_competencia = df[df.competencia == False]
lista_nombre_hoteles = df.nombre_hotel.unique().tolist()
lista_ids_hoteles = df_no_competencia.id_hotel.unique().tolist()
dict_id_nombre_hotel = {'id_hotel': [], 
                        'nombre_hotel': []}
for hotel in lista_nombre_hoteles[1:]:
    dict_id_nombre_hotel["nombre_hotel"].append(hotel)
for id in lista_ids_hoteles:
    dict_id_nombre_hotel["id_hotel"].append(id)
dict_id_nombre_hotel

df_id_hoteles_nombre_hoteles = pd.DataFrame(dict_id_nombre_hotel)

In [9]:
#Relleno la información del id de los hoteles correctos en el df original
sl.incorporar_información_df_original(df,df_id_hoteles_nombre_hoteles, 'nombre_hotel', 'id_hotel', 'id_hotel',df['competencia']==False )

In [10]:
#Calculo la valoración para cada hotel
df["estrellas"] = df["estrellas"].astype('Int64') #transformo la columna estrellas a tipo entero:
df_valoracion = df_no_competencia.groupby('nombre_hotel')['estrellas'].mean().round(1) #creo un nuevo df para calcular la valoración de cada hotel según la media de las estrellas
df_valoracion= df_valoracion.reset_index()

In [11]:
# Relleno la información de la valoración de los hoteles propios con la media de las estrellas calculadas:
sl.incorporar_información_df_original(df,df_valoracion, 'nombre_hotel', 'estrellas', 'valoracion',df['competencia']==False )

In [12]:
#Calculo el precio medio de la noche teniendo en cuenta tanto el nombre como la fecha de la reserva:
df_precios_nulos_hoteles_propios = df_no_competencia[df_no_competencia.precio_noche.isnull()].groupby(['nombre_hotel', 'fecha_reserva'])['id_reserva'].count().reset_index()
df_precios_nulos_hoteles_propios["precio_noche_medio"] = None
for indice in range(0,len(df_precios_nulos_hoteles_propios)):
    nombre_hotel= df_precios_nulos_hoteles_propios.iloc[indice,:].nombre_hotel
    fecha_reserva=df_precios_nulos_hoteles_propios.iloc[indice,:].fecha_reserva
    df_precios_nulos_hoteles_propios.iloc[indice,3] = df[df.nombre_hotel == nombre_hotel][df[df.nombre_hotel == nombre_hotel].fecha_reserva == fecha_reserva].precio_noche.mean().round(2)

#Obtengo los indices de los que tienen precios nulos y son hoteles propios:
lista_indices_precios_nulos_hoteles_propios = df_no_competencia[df_no_competencia.precio_noche.isnull()].index

#Sustituyo los precios que falta en el DF original teniendo en cuenta los precios medios calculados:
for indice in lista_indices_precios_nulos_hoteles_propios:
    fila = df.iloc[indice,:]
    nombre_hotel = fila['nombre_hotel']
    fecha_reserva = fila['fecha_reserva']
    precio_medio = df_precios_nulos_hoteles_propios[(df_precios_nulos_hoteles_propios['nombre_hotel'] == nombre_hotel) & (df_precios_nulos_hoteles_propios['fecha_reserva'] == fecha_reserva)].precio_noche_medio
    df.iloc[indice,10]=precio_medio


In [14]:
# Relleno los datos faltantes en el DF de la competencia conseguidas con el escrapeo
RUTA_SERVICE = os.getenv("RUTA_SERVICE")
URL_ESCRAPEO = os.getenv("URL_ESCRAPEO")
ARCHIVO_GUARDAR_ESCRAPEO= os.getenv("ARCHIVO_GUARDAR_ESCRAPEO")
df_escrapeo = se.escrapeo(URL_ESCRAPEO,RUTA_SERVICE,ARCHIVO_GUARDAR_ESCRAPEO)

In [15]:
#Creo un df con el nombre de los hoteles de la competencia y el id:
df_competencia = df[df.competencia == True] 
values = list(df_competencia.id_hotel.drop_duplicates())
keys = list(df_escrapeo.nombre_hotel.unique())
diccionario_idhotel_nombrehotel = dict(zip(keys,values))
df_idhotel_nombrehotel_competencia = pd.DataFrame(list(diccionario_idhotel_nombrehotel.items()), columns=['nombre_hotel', 'id_hotel'])

In [16]:
#Incorpor el id al df del escrapeo
sl.incorporar_información_df_original(df_escrapeo,df_idhotel_nombrehotel_competencia, 'nombre_hotel', 'id_hotel', 'id_hotel')

In [17]:
#Renombro las columnas del df_escrapeo:
df_escrapeo=df_escrapeo.rename(columns={
    'rating': 'valoracion',
    'fecha_escrapeo': 'fecha_reserva'
})

In [18]:
#Incorporo la información del df del escrapeo al df original
for columna in df_escrapeo.columns[:-1]:
    sl.incorporar_información_df_original(df,df_escrapeo, 'id_hotel', columna, columna,df['competencia']==True )

c:\Formación\DataAnalytics\BootCampDataAnalyst\Proyectos\ProyectoETL\Proyecto_ETL\notebooks\..\src\soporte_limpieza.py:86: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['268' '110' '181' ... '107' '119' '110']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe_a_rellenar.loc[filtro_df_original,columna_a_rellenar] = dataframe_a_rellenar[columna_union].map(diccionario_creado)


In [19]:
#Elimino las columnas que no necesito
df.drop(columns=['id_cliente', 'estrellas','nombre_apellido'],inplace=True)


In [20]:
#Guardo el df como pickle:
df.to_pickle(ARCHIVO_SALIDA)